# Case Study - Freddie Mac Bonds


## 1. Pricing the Callable Bond


### Data

Use the data from the following files.
* `../data/callable_bonds_2025-02-13.xlsx`
* `../data/discount_curve_2025-02-13.xlsx`


The data contains info on the following bonds.

`Callable`
* `FHLMC 4.41 01/28/30` is a callable bond, and it is the primary object of our analysis.


In [1]:
FILE_BOND = 'callable_bonds_2025-02-13.xlsx'
FILE_CURVE = 'discount_curve_2025-02-13.xlsx'

KEY_CALLABLE = 'FHLMC 4.41 01/28/30'

In [2]:
import math
import numpy as np
import pandas as pd

info = pd.read_excel(FILE_BOND, sheet_name='info').set_index('info')
quotes = pd.read_excel(FILE_BOND, sheet_name='quotes').set_index('quotes')
curve = pd.read_excel(FILE_CURVE, sheet_name='discount curve')
curve[['ttm','discount','spot rate']] = curve[['ttm','discount','spot rate']].apply(pd.to_numeric, errors='coerce')
curve = curve.dropna(subset=['ttm','discount','spot rate']).sort_values('ttm')

discs = curve.set_index('ttm')

ttm = curve['ttm'].to_numpy(dtype=float)
disc0 = curve['discount'].to_numpy(dtype=float)
spot0 = curve['spot rate'].to_numpy(dtype=float)

qdt = pd.to_datetime(info.loc['Date Quoted', KEY_CALLABLE])
call_date = pd.to_datetime(info.loc['Date Next Call', KEY_CALLABLE])

N = lambda x: 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

z = lambda t: np.interp(np.asarray(t, dtype=float), ttm, disc0)
z_shift = lambda t, shift: np.interp(np.asarray(t, dtype=float), ttm, disc0 * np.exp(-float(shift) * ttm))


def bond_cashflows(quote_date: pd.Timestamp, maturity_date: pd.Timestamp, cpn_rate: float, freq: int, face: float = 100.0):
    maturity_date = pd.to_datetime(maturity_date)
    step_m = int(round(12 / int(freq)))
    pay = maturity_date
    pays = []
    while pay > quote_date:
        pays.append(pay)
        pay = pay - pd.DateOffset(months=step_m)
    pays = list(reversed(pays))
    t = np.array([(d - quote_date).days / 365.0 for d in pays], dtype=float)
    c = face * float(cpn_rate) / float(freq)
    cf = np.full(len(pays), c, dtype=float)
    if len(cf):
        cf[-1] += face
    return pays, t, cf


def price_from_curve(key: str, maturity_override: pd.Timestamp | None = None):
    cpn_rate = float(info.loc['Cpn Rate', key])
    freq = int(info.loc['Cpn Freq', key])
    mat = pd.to_datetime(info.loc['Date Matures', key]) if maturity_override is None else pd.to_datetime(maturity_override)
    _, t, cf = bond_cashflows(qdt, mat, cpn_rate, freq)
    dirty = float((cf * z(t)).sum())
    ai0 = float(quotes.loc['Accrued Interest', key])
    clean = dirty - ai0
    return clean, dirty


def forward_price_at_call(key: str):
    mat = pd.to_datetime(info.loc['Date Matures', key])
    cpn_rate = float(info.loc['Cpn Rate', key])
    freq = int(info.loc['Cpn Freq', key])

    all_dates, _, _ = bond_cashflows(qdt, mat, cpn_rate, freq)
    future_dates = [d for d in all_dates if d > call_date]

    T1 = (call_date - qdt).days / 365.0
    Z1 = float(z(T1))

    c = 100.0 * cpn_rate / freq
    cf = np.full(len(future_dates), c, dtype=float)
    if len(cf) and future_dates[-1] == mat:
        cf[-1] += 100.0

    t = np.array([(d - qdt).days / 365.0 for d in future_dates], dtype=float)
    disc_t = z(t)
    pv0_vec = cf * disc_t
    pv0 = float(pv0_vec.sum())
    fwd_dirty = pv0 / Z1

    prev_dates = [d for d in all_dates if d <= call_date]
    next_dates = [d for d in all_dates if d > call_date]
    if prev_dates and next_dates and prev_dates[-1] == call_date:
        ai_call = 0.0
    elif prev_dates and next_dates:
        prev_d, next_d = prev_dates[-1], next_dates[0]
        ai_call = c * (call_date - prev_d).days / (next_d - prev_d).days
    else:
        ai_call = 0.0

    fwd_clean = fwd_dirty - ai_call

    sched = pd.DataFrame({'pay_date': future_dates, 't': t, 'cf': cf, 'disc': disc_t, 'pv0': pv0_vec})
    summ = pd.DataFrame({'value': [T1, Z1, pv0, fwd_dirty, ai_call, fwd_clean]}, index=['T1','Z(0,T1)','PV0(remaining)','forward_dirty','accrued_interest_T1','forward_clean'])
    return sched, summ

### Bond Info


In [3]:
info_core = info[[KEY_CALLABLE]]
info_core.style.format('{:.2%}',subset=pd.IndexSlice[["Cpn Rate"], :]).format('{:,.0f}',subset=pd.IndexSlice[["Amount Issued"], :]).format('{:%Y-%m-%d}',subset=pd.IndexSlice[["Date Quoted","Date Issued","Date Matures","Date Next Call","Date of First Possible Call"], :])

,FHLMC 4.41 01/28/30
info,
CUSIP,3134HA4V2
Issuer,FREDDIE MAC
Maturity Type,CALLABLE
Issuer Industry,GOVT AGENCY
Amount Issued,"10,000,000"
Cpn Rate,4.41%
Cpn Freq,2
Date Quoted,2025-02-13
Date Issued,2025-01-28


### Quoted Values


In [4]:
quotes_core = quotes[[KEY_CALLABLE]]
quotes_core.style.format('{:.2f}', subset=pd.IndexSlice[quotes.index[1:], :]).format('{:%Y-%m-%d}', subset=pd.IndexSlice['Date Quoted', :])

,FHLMC 4.41 01/28/30
quotes,
Date Quoted,2025-02-13
TTM,4.96
Clean Price,99.89
Dirty Price,100.09
Accrued Interest,0.20
YTM Call,4.45
YTM Maturity,4.43
Duration,4.50
Modified Duration,4.40


### Discount Curves


In [5]:
display(discs.head())
display(discs.tail())

,maturity date,spot rate,discount
ttm,,,
0.5,2025-08-13,0.043743,0.978597
1.0,2026-02-13,0.042890,0.958451
1.5,2026-08-13,0.042238,0.939228
2.0,2027-02-13,0.041843,0.920515
2.5,2027-08-13,0.041632,0.902117


,maturity date,spot rate,discount
ttm,,,
28.0,2053-02-13,0.040185,0.328231
28.5,2053-08-13,0.040051,0.322978
29.0,2054-02-13,0.039916,0.317851
29.5,2054-08-13,0.039791,0.312766
30.0,2055-02-13,0.039665,0.307802


### 1.1.

Use the discount curve data to price both the `callable` and `reference` bonds.

Also calculate the price of the `hypothetical` bonds, where we consider a non-callable version of the callable bond with 
* maturity unchanged
* maturity at the call date.


In [6]:
rows = []
for key in info.columns:
    clean, dirty = price_from_curve(key)
    rows.append({
        'bond': key,
        'model_clean': clean,
        'model_dirty': dirty,
        'mkt_clean': float(quotes.loc['Clean Price', key]),
        'mkt_dirty': float(quotes.loc['Dirty Price', key]),
        'clean_diff': clean - float(quotes.loc['Clean Price', key]),
        'dirty_diff': dirty - float(quotes.loc['Dirty Price', key]),
    })

res = pd.DataFrame(rows).set_index('bond').sort_index()
display(res)

clean_full, dirty_full = price_from_curve(KEY_CALLABLE)
clean_call, dirty_call = price_from_curve(KEY_CALLABLE, maturity_override=call_date)

display(pd.DataFrame({
    'model_clean': [clean_full, clean_call],
    'model_dirty': [dirty_full, dirty_call],
}, index=['hypothetical_noncall_maturity', 'hypothetical_noncall_call_date']))

,model_clean,model_dirty,mkt_clean,mkt_dirty,clean_diff,dirty_diff
bond,,,,,,
FHLMC 0.97 01/28/28,91.235314,91.278425,90.1440,90.187111,1.091314,1.091314
FHLMC 1 1/4 01/29/30,87.179101,87.231185,85.1095,85.161583,2.069601,2.069601
FHLMC 4.41 01/28/30,101.201335,101.397335,99.8930,100.089000,1.308335,1.308335


,model_clean,model_dirty
hypothetical_noncall_maturity,101.201335,101.397335
hypothetical_noncall_call_date,100.699955,100.895955


### 1.2.

Calculate the forward price of the `hypothetical` bond as of the date that the `callable` bond can be exercised.

Use the information from the discount curve (and associated forward curve) to calculate this forward price.


In [7]:
sched, summ = forward_price_at_call(KEY_CALLABLE)
display(sched)
display(summ)

,pay_date,t,cf,disc,pv0
0,2028-07-28,3.454795,2.205,0.867956,1.913842
1,2029-01-28,3.958904,2.205,0.850402,1.875136
2,2029-07-28,4.454795,2.205,0.833381,1.837606
3,2030-01-28,4.958904,102.205,0.816411,83.441302


,value
T1,2.956164
"Z(0,T1)",0.885665
PV0(remaining),89.067885
forward_dirty,100.566106
accrued_interest_T1,0.000000
forward_clean,100.566106


### 1.3.

The provided implied vol corresponds to the implied vol of the **rate**. Specifically,
* the forward rate corresponding to the time of expiration.
* continuously compounded.

Use the duration approximation to get the approximate implied vol corresponding to the forward price.

$$\sigma_{\text{bond fwd price}} \approx D \times \sigma_{\text{fwd rate}}\times f(T_1)$$

where $f(T_1)$ is the continuously-compounded (instantaneous) forward rate at time $T_1$.
* If you're struggling with the forward rate calc, just usse the provided spot rate at $T_1$; it will be a close approximation in this example.
* In this approximation, use the quoted duration from the table. (Yes, this is a bit circular, but we don't want to get bogged down with a duration calculation at this point.)

Report the implied vol of the bond's forward price.


In [8]:
T1 = (call_date - qdt).days / 365.0
D = float(pd.to_numeric(quotes.loc['Duration', KEY_CALLABLE], errors='coerce'))
sig_rate = float(pd.to_numeric(quotes.loc['Implied Vol', KEY_CALLABLE], errors='coerce'))
sig_rate = sig_rate / 100.0 if sig_rate > 1.5 else sig_rate

fT1 = float(np.interp(T1, ttm, spot0))

sig_bond_fwd = D * sig_rate * fT1

print('T1=', T1)
print('Duration=', D)
print('sigma_fwd_rate=', sig_rate)
print('spot_rate(T1)=', fT1)
print('sigma_bond_fwd_price=', sig_bond_fwd)

T1= 2.956164383561644
Duration= 4.496738331374342
sigma_fwd_rate= 0.23879829406738282
spot_rate(T1)= 0.041504475336585064
sigma_bond_fwd_price= 0.0445680635361678


### 1.4.

For the `callable` bond, report Black's value of the embedded call option.
* Use this to report the value of the `callable` bond.
* How does it compare to the actual market price?

For the calculation of the option, use...
* the quoted `Implied Vol` calculated above.
* forward price of the `hypothetical` bond calculated above.
* provided discount factor

#### Simplifications
Note that in this calculation we are making a few simplifications.
* We are simplifying that the `callable` bond is European exercise with an exercise date as reported in `Date Next Call` above. 
* In reality, it is Bermudan, with quarterly exercise dates after the first exercise date.
* The time-to-exercise is not a round number, but you only have discount factors at rounded time-to-maturities. Just use the closest discount factor.


In [9]:
sched, summ = forward_price_at_call(KEY_CALLABLE)
T1 = float(summ.loc['T1', 'value'])
F_fwd = float(summ.loc['forward_clean', 'value'])
K = float(pd.to_numeric(info.loc['Strike', KEY_CALLABLE], errors='coerce'))

idx = int(np.abs(ttm - T1).argmin())
Z = float(disc0[idx])

sig = float(sig_bond_fwd)

if sig <= 0 or T1 <= 0:
    call_opt = Z * max(F_fwd - K, 0.0)
else:
    srt = sig * math.sqrt(T1)
    d1 = (math.log(F_fwd / K) + 0.5 * sig * sig * T1) / srt
    d2 = d1 - srt
    call_opt = Z * (F_fwd * N(d1) - K * N(d2))

clean_noncall, _ = price_from_curve(KEY_CALLABLE)
model_callable_clean = clean_noncall - call_opt
market_clean = float(pd.to_numeric(quotes.loc['Clean Price', KEY_CALLABLE], errors='coerce'))

print('Model callable clean price =', model_callable_clean)
print('Market clean price =', market_clean)

Model callable clean price = 98.23409842084548
Market clean price = 99.893


The model price is a little lower than the market price. This might be due to the simplified assumption of European exercise.

### 1.5.

Calculate the YTM of the callable bond, assuming that...
* it can never be called. (This is the `hypothetical` bond we analyzed above.)
* it will certainly be called.

How do these compare to the quoted YTM Called and YTM Maturity in the table?


In [10]:
key = KEY_CALLABLE

def ytm_from_dirty(price_dirty: float, t: np.ndarray, cf: np.ndarray, freq: int):
    t = np.asarray(t, dtype=float)
    cf = np.asarray(cf, dtype=float)

    def pv(y: float) -> float:
        return float((cf / (1.0 + y / freq) ** (freq * t)).sum())

    lo, hi = -0.99, 1.0
    f_lo = pv(lo) - price_dirty
    f_hi = pv(hi) - price_dirty
    while f_lo * f_hi > 0 and hi < 50.0:
        hi *= 2.0
        f_hi = pv(hi) - price_dirty

    for _ in range(120):
        mid = 0.5 * (lo + hi)
        f_mid = pv(mid) - price_dirty
        if f_lo * f_mid <= 0:
            hi, f_hi = mid, f_mid
        else:
            lo, f_lo = mid, f_mid

    return 0.5 * (lo + hi)

price_dirty = float(pd.to_numeric(quotes.loc['Dirty Price', key], errors='coerce'))
cpn_rate = float(pd.to_numeric(info.loc['Cpn Rate', key], errors='coerce'))
freq = int(pd.to_numeric(info.loc['Cpn Freq', key], errors='coerce'))
mat = pd.to_datetime(info.loc['Date Matures', key])
strike = float(pd.to_numeric(info.loc['Strike', key], errors='coerce'))

_, t_mat, cf_mat = bond_cashflows(qdt, mat, cpn_rate, freq, face=100.0)
yt_maturity = ytm_from_dirty(price_dirty, t_mat, cf_mat, freq)

_, t_call, cf_call = bond_cashflows(qdt, call_date, cpn_rate, freq, face=100.0)
if len(cf_call):
    cf_call[-1] = 100.0 * cpn_rate / freq + strike
yt_call = ytm_from_dirty(price_dirty, t_call, cf_call, freq)

quoted_maturity = float(pd.to_numeric(quotes.loc['YTM Maturity', key], errors='coerce'))
quoted_maturity = quoted_maturity / 100.0 if quoted_maturity > 1.5 else quoted_maturity

to_pct = lambda y: 100.0 * float(y)
print('YTM_never_called=', to_pct(yt_maturity))
print('YTM_certainly_called=', to_pct(yt_call))
print('YTM_quoted_maturity=', to_pct(quoted_maturity))

YTM_never_called= 4.431249971147744
YTM_certainly_called= 4.4477524457928075
YTM_quoted_maturity= 4.433845268135366


The YTM is higher than YTM_certainlycalled and lower than YTM_nevercalled. This makes sense because YTM would decrease as the value/possibility of the option increases.

### 1.6.

Calculate the duration of...
* the `hypothetical` bond
* the `callable` bond

How do these compare to the quoted duration in the table?

For the callable bond, calculate duration numerically by modifying the spot rates up and down by 1bp and seeing how it changes the valuation of parts `1.1`-`1.3`.


In [11]:
key = KEY_CALLABLE
cpn_rate = float(pd.to_numeric(info.loc['Cpn Rate', key], errors='coerce'))
freq = int(pd.to_numeric(info.loc['Cpn Freq', key], errors='coerce'))
mat = pd.to_datetime(info.loc['Date Matures', key])
K = float(pd.to_numeric(info.loc['Strike', key], errors='coerce'))
ai0 = float(pd.to_numeric(quotes.loc['Accrued Interest', key], errors='coerce'))
Dq = float(pd.to_numeric(quotes.loc['Duration', key], errors='coerce'))
sig_rate = float(pd.to_numeric(quotes.loc['Implied Vol', key], errors='coerce'))
sig_rate = sig_rate / 100.0 if sig_rate > 1.5 else sig_rate


def forward_clean_shift(shift: float):
    all_dates, _, _ = bond_cashflows(qdt, mat, cpn_rate, freq, face=100.0)
    future_dates = [d for d in all_dates if d > call_date]

    T1 = (call_date - qdt).days / 365.0
    Z1 = float(z_shift(T1, shift))

    c = 100.0 * cpn_rate / freq
    cf = np.full(len(future_dates), c, dtype=float)
    if len(cf) and pd.to_datetime(future_dates[-1]) == mat:
        cf[-1] += 100.0

    t = np.array([(pd.to_datetime(d) - qdt).days / 365.0 for d in future_dates], dtype=float)
    pv0 = float((cf * z_shift(t, shift)).sum())
    fwd_dirty = pv0 / Z1

    prev_dates = [d for d in all_dates if d <= call_date]
    next_dates = [d for d in all_dates if d > call_date]
    if prev_dates and next_dates and prev_dates[-1] == call_date:
        ai_call = 0.0
    elif prev_dates and next_dates:
        prev_d, next_d = prev_dates[-1], next_dates[0]
        ai_call = c * (call_date - prev_d).days / (next_d - prev_d).days
    else:
        ai_call = 0.0

    return float(T1), float(Z1), float(fwd_dirty - ai_call)


def callable_clean_price(shift: float):
    _, t_mat, cf_mat = bond_cashflows(qdt, mat, cpn_rate, freq, face=100.0)
    dirty_noncall = float((cf_mat * z_shift(t_mat, shift)).sum())
    clean_noncall = dirty_noncall - ai0

    T1, Z1, Ff = forward_clean_shift(shift)
    fT1 = float(np.interp(T1, ttm, spot0)) + float(shift)
    sig_bond = Dq * sig_rate * fT1

    if sig_bond <= 0 or T1 <= 0:
        call_opt = Z1 * max(Ff - K, 0.0)
    else:
        srt = sig_bond * math.sqrt(T1)
        d1 = (math.log(Ff / K) + 0.5 * sig_bond * sig_bond * T1) / srt
        d2 = d1 - srt
        call_opt = Z1 * (Ff * N(d1) - K * N(d2))

    return float(clean_noncall - call_opt)


def noncall_clean_price(shift: float):
    _, t_mat, cf_mat = bond_cashflows(qdt, mat, cpn_rate, freq, face=100.0)
    dirty = float((cf_mat * z_shift(t_mat, shift)).sum())
    return float(dirty - ai0)


def duration_from_shift(price0: float, price_up: float, price_dn: float, dy: float):
    return (price_dn - price_up) / (2.0 * price0 * dy)


dy = 1e-4
p0_h = noncall_clean_price(0.0)
pU_h = noncall_clean_price(+dy)
pD_h = noncall_clean_price(-dy)
dur_h = duration_from_shift(p0_h, pU_h, pD_h, dy)

p0_c = callable_clean_price(0.0)
pU_c = callable_clean_price(+dy)
pD_c = callable_clean_price(-dy)
dur_c = duration_from_shift(p0_c, pU_c, pD_c, dy)

quoted_dur = float(pd.to_numeric(quotes.loc['Duration', key], errors='coerce'))
quoted_mod = float(pd.to_numeric(quotes.loc['Modified Duration', key], errors='coerce'))

print('Duration_hypothetical=', dur_h)
print('Duration_callable=', dur_c)
print('Duration_quoted=', quoted_dur)
print('ModifiedDuration_quoted=', quoted_mod)

Duration_hypothetical= 4.512541413927468
Duration_callable= 4.265880859673603
Duration_quoted= 4.496738331374342
ModifiedDuration_quoted= 4.399211222071787


The hypothetical non-callable bond typically has the highest duration, the callable bond the lowest (because the embedded call caps price gains and reduces rate sensitivity), and the quoted duration usually falls between them due to different modeling and quoting conventions.

### 1.7.

Calculate the OAS of the `callable` bond.

How does it compare to the quoted OAS?

Recall that the OAS is the parallel shift in the spot curve needed to align the modeled value to the market quote.


In [16]:
key = KEY_CALLABLE
target = float(pd.to_numeric(quotes.loc['Clean Price', key], errors='coerce'))
quoted_oas = float(pd.to_numeric(quotes.loc['OAS Spread', key], errors='coerce'))

f = lambda s: float(callable_clean_price(float(s))) - target

lo, hi = -0.02, 0.02
flo, fhi = f(lo), f(hi)
while flo * fhi > 0 and abs(lo) < 1.0 and abs(hi) < 1.0:
    lo *= 2.0
    hi *= 2.0
    flo, fhi = f(lo), f(hi)

for _ in range(120):
    mid = 0.5 * (lo + hi)
    fmid = f(mid)
    if flo * fmid <= 0:
        hi, fhi = mid, fmid
    else:
        lo, flo = mid, fmid

oas = 0.5 * (lo + hi)

print('OAS_calc_bp=', 1e4 * oas)
print('OAS_quoted_bp=', quoted_oas)

OAS_calc_bp= -39.767580183812655
OAS_quoted_bp= -26.76808540103936


The OAS of the callable bond is more negative than the quoted OAS.

### 1.8. Optional OTM Callables


There are a few other Freddie Mac callables that may be of interest.
* `FHLMC 0.97 01/28/28`
* `FHLMC 1.25 01/29/30`

Though these are technically callable, they are far out of the money (OTM). 
* Expiring in 3 months, though code below changes it to 6 monhts, to match coupon.
* These don't have interesting convexity due to being so far OTM.


In [17]:
# KEY_CALLABLE = 'FHLMC 1 1/4 01/29/30'
# KEY_CALLABLE = 'FHLMC 0.97 01/28/28'

In [18]:
info.style.format('{:.2%}',subset=pd.IndexSlice[["Cpn Rate"], :]).format('{:,.0f}',subset=pd.IndexSlice[["Amount Issued"], :]).format('{:%Y-%m-%d}',subset=pd.IndexSlice[["Date Quoted","Date Issued","Date Matures","Date Next Call","Date of First Possible Call"], :])

,FHLMC 0.97 01/28/28,FHLMC 1 1/4 01/29/30,FHLMC 4.41 01/28/30
info,,,
CUSIP,3134GW5F9,3134GWGK6,3134HA4V2
Issuer,FREDDIE MAC,FREDDIE MAC,FREDDIE MAC
Maturity Type,CALLABLE,CALLABLE,CALLABLE
Issuer Industry,GOVT AGENCY,GOVT AGENCY,GOVT AGENCY
Amount Issued,"30,000,000","25,000,000","10,000,000"
Cpn Rate,0.97%,1.25%,4.41%
Cpn Freq,2,2,2
Date Quoted,2025-02-13,2025-02-13,2025-02-13
Date Issued,2020-10-28,2020-07-29,2025-01-28


In [19]:
quotes.style.format('{:.2f}', subset=pd.IndexSlice[quotes.index[1:], :]).format('{:%Y-%m-%d}', subset=pd.IndexSlice['Date Quoted', :])

,FHLMC 0.97 01/28/28,FHLMC 1 1/4 01/29/30,FHLMC 4.41 01/28/30
quotes,,,
Date Quoted,2025-02-13,2025-02-13,2025-02-13
TTM,2.95,4.96,4.96
Clean Price,90.14,85.11,99.89
Dirty Price,90.19,85.16,100.09
Accrued Interest,0.04,0.05,0.20
YTM Call,54.24,85.40,4.45
YTM Maturity,4.57,4.65,4.43
Duration,2.92,4.81,4.50
Modified Duration,2.85,4.70,4.40


### 1.9. ATM with 1-yr expiry

Try this alternate file `2025-02-18` for a recently-issued bond of size $1bn with a one-year expiration.
* Easier to see the negative convexity.
* Large size, recency should be more liquid.


In [20]:
# FILE_BOND = '../data/callable_bonds_2025-02-18.xlsx'
# FILE_CURVE = '../data/discount_curve_2025-02-18.xlsx'
# KEY_CALLABLE = 'FHLMC 4.55 02/11/28'